<a href="https://colab.research.google.com/github/pansplawik/eegParkinsonCognitiveDisorders/blob/master/secondVersion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mne

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 59.9 MB/s eta 0:00:00


In [15]:
import mne
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
import csv

In [3]:
labels = pd.read_csv('/content/drive/MyDrive/dyplom/dataset.csv', sep=';')
path='/content/drive/MyDrive/dyplom/EEG'

In [5]:
path='/content/drive/MyDrive/dyplom/EEG'
def load_epochs_from_dir(path, labels):
    epochs_array = []
    for filename, label in zip(os.listdir(path), labels['PD']):
        print(filename)
        if filename.endswith('.edf'):
            filepath = os.path.join(path, filename)
            raw = mne.io.read_raw_edf(filepath, preload=True)
            annotations = raw.annotations
            change=0
            for i in range(len(annotations)):
              if annotations.description[i] == "break":
                annotations.description[i] = "BAD" 
                change=change+1
              elif annotations.description[i] == 'Break end':
                annotations.description[i] = "BAD"
                change=change+1
              elif annotations.description[i] == 'Miesnie':
                annotations.description[i] = "BAD"
                change=change+1
              elif annotations.description[i] == 'Break start':
                annotations.description[i] = "BAD"
                change=change+1
              elif annotations.description[i] == 'Ruch':
                annotations.description[i] = "BAD"
                change=change+1
            print(change)
            raw.set_annotations(annotations)
            raw.filter(1, 40, fir_design='firwin')
            events = mne.make_fixed_length_events(raw, duration=1.0)
            epochs = mne.Epochs(raw, events, tmin=0, tmax=2, baseline=(0,2), detrend=1, reject_by_annotation='BAD',preload=True)
            epochs_array.append([epochs, label])
            print(epochs.drop_log)
            del epochs
            del raw
            del annotations
    return epochs_array


In [6]:
def psds(epochs_array, labels):
    data = []
    for epochs, label in zip(epochs_array, labels):
        epo_spectrum = epochs.compute_psd(method='multitaper', fmin=8, fmax=13, picks='eeg')
        psds, freqs = epo_spectrum.get_data(return_freqs=True)
        for p in psds:
            data.append([p, label])
        del psds
        del freqs
    return data

In [7]:
data = load_epochs_from_dir('/content/drive/MyDrive/dyplom/EEG',labels)
df = pd.DataFrame(data, columns=['value', 'label'])
df.to_csv('data.csv', index=False)

PD130.edf
Extracting EDF parameters from /content/drive/MyDrive/dyplom/EEG/PD130.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 658943  =      0.000 ...  1286.998 secs...
12
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 1691 samples (3.303 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s


Not setting metadata
1287 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 1287 events and 1025 original time points ...


[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.5s finished


39 bad epochs dropped
((), (), (), (), (), (), (), ('BAD',), ('BAD',), ('BAD',), ('BAD',), ('BAD',), ('BAD',), ('BAD',), ('BAD',), ('BAD',), ('BAD',), ('BAD',), ('BAD',), ('BAD',), ('BAD',), ('BAD',), ('BAD',), ('BAD',), ('BAD',), ('BAD',), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), ('BAD',), ('BAD',), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), ('BAD',), ('BAD',), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (),

...

104 bad epochs dropped
((), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), ('BAD',), ('BAD',), (), (), (), (), ('BAD',), ('BAD',), ('BAD',), ('BAD',), ('BAD',), ('BAD',), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), ('BAD',), ('BAD',), (), ('BAD',), ('BAD',), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), ('BAD',), ('BAD',), (), (), ('BAD',), ('BAD',), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), 

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s


Not setting metadata
1260 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 1260 events and 1025 original time points ...


[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.5s finished


42 bad epochs dropped
((), (), (), (), (), ('BAD',), ('BAD',), ('BAD',), ('BAD',), ('BAD',), ('BAD',), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), ('BAD',), ('BAD',), (), (), ('BAD',), ('BAD',), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), ('BAD',), ('BAD',), (), (), (), (), (), (), (), (), ('BAD',), ('BAD',), (), (), (), (), (), (), (), ('BAD',), ('BAD',), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (

In [8]:
psd = psds([epochs for epochs, label in data], [label for epochs, label in data])
df = pd.DataFrame(psd, columns=['psds', 'label'])
df.to_csv('psd.csv', index=False)

    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spe

In [9]:
X_train, X_test, y_train, y_test = train_test_split([x[0] for x in psd], [x[1] for x in psd], test_size=0.2, random_state=42)
X_train_2d = np.array([x[0] for x in X_train]).reshape(len(X_train), -1)
X_test_2d = np.array([x[0] for x in X_test]).reshape(len(X_test), -1)
y_train=np.array(y_train)
y_train=y_train.transpose()
y_test=np.array(y_test)
y_test=y_test.transpose()
svm = SVC(kernel='linear', C=1, random_state=42)

In [21]:
nb_classifier = GaussianNB()
nb_classifier.fit(X_train_2d, y_train)

# Predykcja na danych testowych
y_pred = nb_classifier.predict(X_test_2d)
# Obliczenie dokładności
accuracy = accuracy_score(y_test, y_pred)
print("Dokładność: ", accuracy)

Dokładność:  0.3022878342865976


In [10]:
print(X_test_2d.shape)
print(X_train_2d.shape)

(25876, 10)
(103502, 10)


In [11]:
svm.fit(X_train_2d, y_train)

SVC(C=1, kernel='linear', random_state=42)

In [12]:
y_pred = svm.predict(X_test_2d)

In [13]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.4390554954397898


In [14]:
from joblib import dump, load
dump(svm, 'drugi.joblib') 

['drugi.joblib']